In [ ]:
%%capture
!pip install datasets deepsig

In [ ]:
from datasets import load_dataset

task_to_df  = {
    "bbbp" : load_dataset("katielink/moleculenet-benchmark", 'bbbp')['test'].to_pandas(),
    "bace" : load_dataset("katielink/moleculenet-benchmark", 'bace')['test'].to_pandas(),
    "esol" : load_dataset("katielink/moleculenet-benchmark", 'esol')['test'].to_pandas(),
    "freesolv" : load_dataset("katielink/moleculenet-benchmark", 'freesolv')['test'].to_pandas(),
    "clintox" : load_dataset("katielink/moleculenet-benchmark", 'clintox')['test'].to_pandas()
}

README.md:   0%|          | 0.00/3.05k [00:00<?, ?B/s]

bbbp/train.csv:   0%|          | 0.00/116k [00:00<?, ?B/s]

bbbp/test.csv:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

bbbp/valid.csv:   0%|          | 0.00/15.7k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

clintox/test.csv:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

clintox/valid.csv:   0%|          | 0.00/10.3k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

In [ ]:
import json

def index(s):
  return int(s.split("_")[-1])


file_paths = ["/content/id_to_pred_mistral-large-2407.json", "/content/id_to_pred_llama-3.1-405b-instruct_final.json", "/content/id_to_pred_gpt-4o.json", "/content/id_to_pred_gemini-1.5-pro-002_final.json"]
responses = {}

for file_path in file_paths:
  with open(file_path, 'r') as f:
    data = json.load(f)
    responses[file_path] = {}
    for task_name in ['bbbp', 'bace', 'clintox', 'esol', 'freesolv']:
      responses[file_path][task_name] = {}
      for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
        responses[file_path][task_name][colname] = [data[k] for k in list(sorted(data.keys(), key = index)) if (k.split("_")[0] == task_name) and (k.split("_")[1] == colname)]

In [ ]:
for file_path in file_paths:
  for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
    task_to_df['bbbp'][f'{file_path}_{colname}_response'] = responses[file_path]['bbbp'][colname]
    task_to_df['bace'][f'{file_path}_{colname}_response'] = responses[file_path]['bace'][colname]
    task_to_df['clintox'][f'{file_path}_{colname}_response'] = responses[file_path]['clintox'][colname]
    task_to_df['esol'][f'{file_path}_{colname}_response'] = responses[file_path]['esol'][colname]
    task_to_df['freesolv'][f'{file_path}_{colname}_response'] = responses[file_path]['freesolv'][colname]

In [ ]:
def extract_boolean_answer(s):
  if "yes" in s.lower().split("decision:")[-1]:
    return True
  elif "no" in s.lower().split("decision:")[-1]:
    return False
  else:
    return False


import re
def extract_float_answer(s):

    if "decision:" in s.lower():
      s = s.lower().split('decision:')[-1]
    elif "decision" in s.lower():
      s = s.lower().split('decision')[-1]

    float_pattern = r'(-?\d+\.?\d*)'

    match = re.search(float_pattern, s)

    if match:
        return float(match.group())
    else:
       return 0


for file_path in file_paths:
  for df in list(task_to_df.values())[:2]: # Classification tasks
    for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
      df[f'{file_path}_{colname}_pred'] = df[f'{file_path}_{colname}_response'].apply(extract_boolean_answer)

for file_path in file_paths:
  for df in list(task_to_df.values())[-1:]: # Classification tasks
    for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
      df[f'{file_path}_{colname}_pred'] = df[f'{file_path}_{colname}_response'].apply(extract_boolean_answer)

for file_path in file_paths:
  for df in list(task_to_df.values())[2:4]: # Regression tasks
    for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
      df[f'{file_path}_{colname}_pred'] = df[f'{file_path}_{colname}_response'].apply(extract_float_answer)


In [ ]:
list(task_to_df.values())[:2]

In [ ]:
# Score each example with accuracy

task_to_label_colname = {
    "bbbp" : "p_np",
    "bace" : "Class",
    "clintox" : "CT_TOX"
}

for file_path in file_paths:
  for k, df in list(task_to_df.items())[:2]: # Classification tasks
    for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
      df[f'{file_path}_{colname}_score'] = df[f'{file_path}_{colname}_pred'] == df[task_to_label_colname[k]]

for file_path in file_paths:
  for k, df in list(task_to_df.items())[-1:]: # Classification tasks
    for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
      df[f'{file_path}_{colname}_score'] = df[f'{file_path}_{colname}_pred'] == df[task_to_label_colname[k]]

In [ ]:
# Score each regression example with negative absolute error

task_to_label_colname = {
    "bbbp": "p_np",
    "bace": "Class",
    "esol": "measured log solubility in mols per litre",
    "freesolv": "expt",
    "clintox": "CT_TOX"
}

for file_path in file_paths:
  for k, df in list(task_to_df.items())[2:4]: # Regression tasks
    for colname in ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']:
      df[f'{file_path}_{colname}_score'] = abs(df[f'{file_path}_{colname}_pred'] - df[task_to_label_colname[k]]) * -1

In [ ]:
task_to_df['esol'].head()

,Compound ID,ESOL predicted log solubility in mols per litre,Minimum Degree,Molecular Weight,Number of H-Bond Donors,Number of Rings,Number of Rotatable Bonds,Polar Surface Area,measured log solubility in mols per litre,smiles,...,/content/id_to_pred_gpt-4o.json_smiles_score,/content/id_to_pred_gpt-4o.json_deepsmiles_score,/content/id_to_pred_gpt-4o.json_selfies_score,/content/id_to_pred_gpt-4o.json_inchi_score,/content/id_to_pred_gpt-4o.json_iupac_score,/content/id_to_pred_gemini-1.5-pro-002_final.json_smiles_score,/content/id_to_pred_gemini-1.5-pro-002_final.json_deepsmiles_score,/content/id_to_pred_gemini-1.5-pro-002_final.json_selfies_score,/content/id_to_pred_gemini-1.5-pro-002_final.json_inchi_score,/content/id_to_pred_gemini-1.5-pro-002_final.json_iupac_score
0,Dieldrin,-4.533,1,380.913,0,5,0,12.53,-6.290,ClC4=C(Cl)C5(Cl)C3C1CC(C2OC12)C3C4(Cl)C5(Cl)Cl,...,-0.290,-1.790,-2.790,-1.290,-1.290,-1.790,-6.290,-1.790,-0.710,-1.290
1,Valeraldehyde,-1.103,1,86.134,0,0,3,17.07,-0.850,CCCCC=O,...,-0.950,-0.350,-0.950,-0.150,-0.050,-1.150,-0.150,-0.650,-0.150,-0.050
2,4-Pentene-1-ol,-0.791,1,86.134,1,0,3,20.23,-0.150,OCCCC=C,...,-1.350,-0.750,-1.350,-1.200,-1.050,-2.750,-3.350,-3.650,-1.350,-0.450
3,brompyrazone,-3.005,1,266.098,1,2,1,60.91,-3.127,c1ccccc1n2ncc(N)c(Br)c2(=O),...,-0.627,-0.373,-0.373,-3.627,-0.627,-0.627,-0.373,-0.127,-0.127,-0.127
4,p-Aminophenol,-1.231,1,109.128,2,1,0,46.25,-0.800,Nc1ccc(O)cc1,...,-1.300,-0.700,-1.700,-0.800,-1.500,-1.300,-0.300,-2.200,-1.300,-1.300


# ASO Significance Testing

In [ ]:
from itertools import product

import numpy as np
from deepsig import multi_aso

seed = 1234
np.random.seed(seed)

scores_by_representation = [list(np.concatenate(
    [task_to_df[task_name][f'{file_path}_{colname}_score'].astype(int) for file_path in file_paths for task_name in
     ['bbbp', 'bace', 'clintox', 'esol', 'freesolv']])) for colname in
                            ['smiles', 'deepsmiles', 'selfies', 'inchi', 'iupac']]

In [ ]:
scores_smiles = scores_by_representation[0]
scores_deepsmiles = scores_by_representation[1]
scores_selfies = scores_by_representation[2]
scores_inchi = scores_by_representation[3]
scores_iupac = scores_by_representation[4]

In [ ]:
from deepsig import aso

eps_min_deepsmiles = aso(scores_deepsmiles, scores_smiles, confidence_level=0.95, seed=seed, num_comparisons=4, show_progress=True)
eps_min_selfies = aso(scores_selfies, scores_smiles, confidence_level=0.95, seed=seed, num_comparisons=4, show_progress=True)
eps_min_inchi = aso(scores_inchi, scores_smiles, confidence_level=0.95, seed=seed, num_comparisons=4, show_progress=True)
eps_min_iupac = aso(scores_iupac, scores_smiles, confidence_level=0.95, seed=seed, num_comparisons=4, show_progress=True)

Bootstrap iterations: 100%|█████████▉| 999/1000 [04:56<00:00,  3.37it/s]


In [ ]:
print(f"Eps min DeepSMILES: {eps_min_deepsmiles}\nEps min SELFIES: {eps_min_selfies}\nEps min InChI: {eps_min_inchi}\nEps min IUPAC: {eps_min_iupac}")

Eps min DeepSMILES: 1.0
Eps min SELFIES: 1.0
Eps min InChI: 0.17387435582083577
Eps min IUPAC: 0.05895998414669419
